# requirements

In [1]:
%%sh
pip install --upgrade langchain
pip install --upgrade openai
pip install --upgrade python-dotenv
pip install --upgrade redis
pip install --upgrade requests
pip install --upgrade tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 330.3 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: openai
    Found existing installation: openai 0.27.5
    Uninstalling openai-0.27.5:
      Successfully uninstalled openai-0.27.5


# environment

In [3]:
%%sh
docker exec redis uname || docker run --rm --name redis -d -p 13333:8001 -p 10001:6379 redis/redis-stack:latest

Linux


# imports

In [4]:
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.redis import Redis
import json
import os
import random
import redis
import requests
import time
import urllib.request

# vectorization

In [14]:
load_dotenv()
embeddings = OpenAIEmbeddings()

index_name = "demo"
Redis.drop_index(index_name=index_name, delete_documents=True)

# traverse the ./content directory recursively and load each file that contains plain text

docs = []
for root, _, files in os.walk("./content"):
    for file in files:
        if file.endswith(".md") or file.endswith(".html") or file.endswith(".json"):
            filepath = os.path.join(root, file)
            print(filepath)
            with open(filepath, "r") as f:
                text = f.read()
                metadata={'source': filepath}
                doc = Document(page_content=text, metadata=metadata)
                docs.append(doc)

rds = Redis.from_documents(docs, embeddings, index_name=index_name)


./content/troubleshooting.md
./content/home.md
./content/get-started.md
./content/guides.md
./content/contribute.md
./content/user-pages.md
./content/apps.html
./content/user-pages/tigerinus.md
./content/contribute/development.md
./content/contribute/wiki-editing-guidelines.md
./content/contribute/alpha.md
./content/guides/move-docker-images-and-volumes-to-a-diffferent-storage.md
./content/guides/running-casaos-on-windows-with-wsl2.md
./content/troubleshooting/appstore-does-not-load.md
./content/appfile/jellyfin.json
./content/apps/jellyfin.html
./content/zh/apps.md
./content/zh/troubleshooting.md
./content/zh/home.md
./content/zh/get-started.md
./content/zh/guides.md
./content/zh/contribute.md
./content/zh/user-pages.md
./content/zh/contribute/development.md


# recommendation

In [15]:
query = "move docker image to another host"
for doc in rds.similarity_search(query):
    print(doc.metadata)
    print(doc.page_content)

{'source': './content/guides/move-docker-images-and-volumes-to-a-diffferent-storage.md'}
---
title: How to move docker images and volumes to a different storage
description: guides
published: true
date: 2023-03-08T21:12:34.543Z
tags: docker
editor: markdown
dateCreated: 2022-08-13T17:00:12.935Z
---

There are few options to change where Docker images and volumes are stored. You can make `/var/lib/docker` a mount point to some new partition; or symbol link `var/lib/docker` to a path on certain filesystems. This approach below involves updating `docker.service` in systemd with the new path.

*(credits: https://linuxconfig.org/how-to-move-docker-s-default-var-lib-docker-to-another-directory-on-ubuntu-debian-linux).*

> Risk of this approach is from time to time, when Docker is upgraded, there is a chance that the updated `docker.service` file gets reverted to default settings. In that case, you will not see the current images and volumes and will have to update the file again
{.is-warning

# QA bot

In [25]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=rds.as_retriever(),
    return_source_documents=True
)


In [31]:
result = qa(
    {
        "query": "How to move docker image?",
    }
)

print(result["result"])
print("")
print(result["source_documents"][0].metadata)

 Steps 1-7 in the article "How to move docker images and volumes to a different storage" provide instructions on how to move docker images.

{'source': './content/guides/move-docker-images-and-volumes-to-a-diffferent-storage.md'}
